In [13]:
%pylab
import pyfits
import scipy
import scipy.ndimage.filters


def ndflux(image,bins=None):
    
    if bins == None:
        bins = .001 * np.size(image)

    h,b = np.histogram(image,bins=bins)
    b = b[:-1] + .5*np.diff(b)
    dflux = b * h    
    ndflux = dflux/np.sum(dflux)
    
    return b,ndflux

def perimeter(image, a, w,ndf=None,b=None):
        
    if ndf is None:
        b,ndf = ndflux(image)
    
    perimeter_bins = b[np.logical_and( (np.cumsum(ndf) > a-w) , ((np.cumsum(ndf) < a+w)) )]
    low = perimeter_bins[0]
    high = perimeter_bins[-1]

    perimeter_x = []
    perimeter_y = []

    for i,row in enumerate(image):
        for j,val in enumerate(row):
            if low <= val <= high:
                perimeter_x.append(i)
                perimeter_y.append(j)
            
    return np.array(perimeter_x), np.array(perimeter_y)

def mean(image):
    
    x,y = image.shape
    
    m,cx,cy = 0.,0.,0.
    
    for xx in range(x):
        for yy in range(y):
            val = image[xx][yy]
            cx += val*xx
            cy += val*yy
            m += val
        
    return cx/m, cy/m
        
def peak(image):
    i = np.argmax(image.flatten())
    lx,ly = image.shape
    return i/(ly),i%(ly)

def r_eff(px,py,cx,cy):
    
    r = np.sqrt((px-cx)**2 + (py-cy)**2)/(len(px))
    return np.sum(r)
    

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [11]:
image = np.copy(pyfits.open("emhalpha_flux.fits")[0].data)

images = [image,
          scipy.ndimage.filters.gaussian_filter(image,[50,50]),
          scipy.ndimage.filters.gaussian_filter(image,[100,100])]

a = np.linspace(.01,.3,25)
w = [.01,.05,.1]

result_a = []
result_r =[]

for img in images:
    b,ndf = ndflux(img)
    cx_m,cy_m = mean(img)
    cx_p,cy_p = peak(img)
    for aa in a:
        for ww in w:
            px,py=perimeter(img,aa,ww,ndf,b)
            result_a.append(aa)
            result_r.append(r_eff(px,py,cx_m,cy_m))
            result_a.append(aa)
            result_r.append(r_eff(px,py,cx_p,cy_p))
    

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:22: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [14]:
scatter(result_a,result_r)

In [17]:
def Rs(Q0,T,ab,nh):
    
    return (3*Q0/(4*np.pi*nh*nh*ab))**(1/3.)